In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
z=pickle.load(open("pickle_model.pickle","rb"))

In [3]:
z

Lasso(alpha=0.001, random_state=0)

In [4]:
test = pd.read_csv('test.csv')

In [5]:
test['Functional'] = test['Functional'].fillna('Typ')

In [6]:
test['Electrical'] = test['Electrical'].fillna('SBrkr')

In [7]:
test['KitchenQual'].fillna('TA', inplace=True)

In [8]:
test["Exterior1st"].fillna(test['Exterior1st'].mode()[0], inplace=True)

In [9]:
test["Exterior2nd"].fillna(test["Exterior2nd"].mode()[0], inplace=True)

In [10]:
test['SaleType'].fillna(test['SaleType'].mode()[0], inplace=True)

In [11]:
test['PoolQC'].fillna('No', inplace=True)

In [12]:
test['Alley'].fillna('No', inplace=True)

In [13]:
test['FireplaceQu'].fillna('No', inplace=True)

In [14]:
test['Fence'].fillna('No', inplace=True)

In [15]:
test['MiscFeature'].fillna('No', inplace=True)

In [16]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    test[col]=test[col].fillna('No')

In [17]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    test[col]=test[col].fillna('No')

In [18]:
for col in ('GarageArea', 'GarageCars'):
    test[col]=test[col].fillna(0)

In [19]:
test.fillna(test.median(), inplace=True)

/var/folders/6f/qftvzclj2fv91pxvhcwx_14m0000gp/T/ipykernel_85186/2976255369.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test.fillna(test.median(), inplace=True)


In [20]:
test['MSZoning'].fillna(test['MSZoning'].mode()[0], inplace=True)

In [21]:
test['Utilities'].fillna(test['Utilities'].mode()[0], inplace=True)

In [22]:
test['MasVnrType'].fillna(test['MasVnrType'].mode()[0],inplace=True)

In [23]:
test['AllSF'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF']

In [24]:
test['BackyardSF'] =  test['LotArea'] - test['1stFlrSF']

In [25]:
test['PorchSF'] = test['WoodDeckSF'] + test['OpenPorchSF'] + test['EnclosedPorch'] + test['3SsnPorch'] + test['ScreenPorch']

In [26]:
test['Total_Bathrooms'] = test['FullBath'] + test['BsmtFullBath'] + (.5*test['BsmtHalfBath']) + (.5*test['HalfBath'])

In [27]:
test['MedNhbdArea'] = test.groupby('Neighborhood')['GrLivArea'].transform('median')

In [28]:
test['IsAbuGr'] = test[['MedNhbdArea', 'GrLivArea']].apply(lambda x: 'yes' if x['GrLivArea']> x['MedNhbdArea'] else 'no', axis=1)

In [29]:
indeces = ['PoolArea',
 'BackyardSF',
 'LotArea',
 'LowQualFinSF',
 'BsmtFinSF2',
 'MasVnrArea',
 'OpenPorchSF',
 'WoodDeckSF',
 'LotFrontage',
 'AllSF',
 '1stFlrSF',
 'BsmtFinSF1',
 'PorchSF',
 'GrLivArea',
 'TotalBsmtSF',
 'MedNhbdArea',
 'BsmtUnfSF',
 '2ndFlrSF']

In [30]:
for x in indeces:
    test[x] = np.log1p(test[x])

In [31]:
scaler=pickle.load(open("scaler_pickle.pickle", "rb"))

In [32]:
scale = ['MedNhbdArea', 'WoodDeckSF','BackyardSF','OpenPorchSF' ,'PorchSF', 'AllSF', '1stFlrSF','2ndFlrSF', 'BsmtFinSF1', 'BsmtFinSF2','BsmtUnfSF','GarageArea','GrLivArea','LotArea','LotFrontage','LowQualFinSF','MasVnrArea','TotalBsmtSF','PoolArea']

In [33]:
testscale=test[scale]

In [34]:
testscale=pd.DataFrame(scaler.transform(testscale), columns=testscale.columns)

In [35]:
encode=list(set(test.columns)-set(scale)-set(['Id']))

In [36]:
testencode = test[encode]

In [37]:
test=testscale.merge(testencode.reset_index(), left_index=True, right_index=True)

In [38]:
test=pd.DataFrame(pd.get_dummies(data=test))

In [39]:
test.shape

(1459, 291)

In [40]:
columns=pickle.load(open("columnssaved.pickle","rb"))

In [41]:
columns

Index(['MedNhbdArea', 'WoodDeckSF', 'BackyardSF', 'OpenPorchSF', 'PorchSF',
       'AllSF', '1stFlrSF', '2ndFlrSF', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'ExterCond_Ex', 'ExterCond_Fa', 'ExterCond_Gd', 'ExterCond_Po',
       'ExterCond_TA', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH',
       'MSZoning_RL', 'MSZoning_RM'],
      dtype='object', length=309)

In [43]:
finaltest=test.reindex(columns=columns, fill_value=0)

In [44]:
finaltest.shape

(1459, 309)

In [45]:
finaltest=finaltest[columns]

In [46]:
finaltest.shape

(1459, 309)

In [47]:
predictions = z.predict(finaltest)

In [48]:
final_preds=np.expm1(predictions)

In [49]:
new_test= pd.read_csv('test.csv')

In [50]:
submission = pd.DataFrame(new_test['Id'], columns=['Id'])

In [51]:
submission['SalePrice'] = final_preds

In [52]:
submission.to_csv('submission_jan_.csv', index=False, header=True)